# **📡 ZEND Customer Support Query Classification System**

## **Problem Statement**

Customer support systems receive thousands of user queries daily related to billing issues, refunds, technical problems, complaints, and product inquiries. 

Manually categorizing these queries into appropriate intents and identifying customer sentiment is time-consuming and inefficient.

The objective of this project is to build a Machine Learning model that can:

- Automatically classify customer queries into predefined intents.
- Detect the sentiment of the query (Positive, Neutral, Negative).
- Help improve automated ticket routing and customer experience.


## Dataset Description

The dataset contains synthetic customer support queries generated for a telecom and cloud services company named "ZEND".

### Features:

- **text** → Customer query text
- **intent** → Category of the query
    - Billing
    - Refund
    - Technical
    - Complaint
    - Product Inquiry
- **sentiment** → Emotional tone of the query
    - Positive
    - Neutral
    - Negative

### Dataset Size:

- Total Records: ~20,000
- Balanced Sentiment Distribution
- Balanced Intent Distribution


## Dataset Generation Process

The dataset was synthetically generated using:

1. Predefined intent templates.
2. Entity injection (product names, regions, pricing, features).
3. Balanced sentiment labeling.
4. Random shuffling to ensure uniform distribution.

No external real customer data was used. The dataset was created for educational and model development purposes.

This approach ensures:

- Controlled class balance
- Scalable data generation
- Structured intent categories


## **Step 1: Data Load**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# For text preprocessing
import re
import string


In [ ]:
df=pd.read_csv("synthetic_customer_support_dataset.csv")

In [ ]:
df.head()

### What Was Done:
- Loaded the dataset.
- Displayed first 5 records to understand structure.

### Observation:
- The dataset contains three main columns:
    - text
    - intent
    - sentiment


In [ ]:
print("Dataset Shape:", df.shape)
print("\nDataset Info:\n")
df.info()


### What Was Done:
- Checked total number of rows and columns.
- Verified data types.

### Key Insights:
- The dataset contains approximately 20,000 records.
- All columns are of type object (text data).
- No unexpected numeric columns.
- Structure is suitable for NLP preprocessing.


In [ ]:
df.isnull().sum()


In [ ]:
df.duplicated().sum()

### What Was Done:
- Checked for missing or null values and duplicates.

### Key Insights:
- No missing values detected.
- Dataset is clean and ready for preprocessing.
- No need for imputation.


In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(x="intent", data=df)
plt.xticks(rotation=45)
plt.title("Intent Distribution")
plt.show()

df["intent"].value_counts()


### What Was Done:
- Analyzed distribution of customer intents.
- Visualized class balance using count plot.

### Key Insights:
- All five intents are nearly equally distributed.
- No class imbalance observed.
- Suitable for multi-class classification.


In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x="sentiment", data=df)
plt.title("Sentiment Distribution")
plt.show()

df["sentiment"].value_counts()


### What Was Done:
- Analyzed sentiment label distribution.
- Checked for class balance.

### Key Insights:
- Positive, Neutral, and Negative classes are evenly distributed.
- Balanced dataset improves model training stability.
- Reduces bias toward majority class.


In [ ]:
length_stats = pd.DataFrame({
    "original_length": df["text"].str.len()
})

length_stats.describe()


### What Was Done:
- Calculated length of each query.
- Analyzed statistical summary of text length.

### Key Insights:
- Queries are short and structured.
- Most customer queries fall within a moderate character range.
- No extremely long or noisy text detected.


## Overall Data Understanding

- The dataset is clean and structured.
- No missing values detected.
- Balanced intent distribution.
- Balanced sentiment distribution.
- Query lengths are consistent and manageable.
- Dataset is ready for preprocessing and feature engineering.




## **Step 2: Data Preprocessing** 



Natural Language Processing (NLP) models cannot directly understand raw text.
Therefore, text preprocessing is required to clean and standardize the data.

## Objectives:

- Convert text to lowercase
- Remove punctuation
- Remove numbers
- Remove extra spaces
- Prepare clean text for vectorization (TF-IDF)

## Why Preprocessing is Important:

- Reduces noise in the dataset
- Ensures consistent word representation
- Improves model accuracy
- Prevents duplicate word variations (e.g., "Bill" and "bill")

This step prepares the dataset for feature extraction.


In [ ]:
import re
import string


In [ ]:
df[["text"]].head()


### Observation Before Cleaning:

- Text contains uppercase letters.
- Punctuation symbols are present.
- Some templates contain numbers (amounts).
- Inconsistent formatting may exist.

We will standardize the text to improve model performance.


In [ ]:
def clean_text(text):
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


### What Was Done:

1. Converted text to lowercase → ensures uniform word comparison.
2. Removed numbers → prevents irrelevant numeric influence.
3. Removed punctuation → avoids unnecessary tokens.
4. Removed extra spaces → improves text consistency.

This prepares text for vectorization.


In [ ]:
df["clean_text"] = df["text"].apply(clean_text)

df[["text", "clean_text"]].head()


### Key Insights After Cleaning:

- All text is now lowercase.
- Special characters and punctuation removed.
- Numeric values removed.
- Text is standardized.

The dataset is now clean and ready for feature extraction (TF-IDF or CountVectorizer).


In [ ]:
length_stats["clean_length"] = df["clean_text"].str.len()

length_stats.describe()

### Length Comparison Insight:

- Cleaned text is slightly shorter due to removal of punctuation and numbers.
- No significant data loss occurred.
- Text meaning remains intact.

The cleaning process reduced noise without affecting semantic structure.


In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in ENGLISH_STOP_WORDS]
    return " ".join(filtered)

df["clean_text_st"] = df["clean_text"].apply(remove_stopwords)


In [ ]:
length_stats['final_len']=df['clean_text_st'].apply(len)

length_stats.describe()

In [ ]:
df

### Stopword Removal Insight:

- Common words like "the", "is", "for" are removed.
- Reduces dimensionality.
- Helps model focus on meaningful keywords.


In [ ]:
pd.crosstab(df["intent"], df["sentiment"])

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def vader_check(text):
    score = analyzer.polarity_scores(text)["compound"]
    if score >= 0.49:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

df["vader_sentiment"] = df["text"].apply(vader_check)

pd.crosstab(df["sentiment"], df["vader_sentiment"])


## Sentiment Label Validation

VADER sentiment analysis was applied to validate the manually assigned sentiment labels.

Results showed approximately 93% agreement between dataset labels and VADER predictions.

Minor disagreements occurred between Neutral and Positive classes, likely due to threshold sensitivity.

This confirms that the dataset labels are logically consistent and reliable for model training.

The original sentiment labels were retained as ground truth for supervised learning.


In [ ]:
df

## **EDA**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")


In [ ]:
# Message Length Feature
df["message_length"] = df["text"].apply(len)

print(df.head())


**1️⃣ UNIVARIATE ANALYSIS**

A) Distribution of Intents

In [ ]:
plt.figure()
df["intent"].value_counts().plot(kind="bar")
plt.title("Distribution of Intents")
plt.xlabel("Intent")
plt.ylabel("Count")
plt.show()


B) Distribution of Sentiments

In [ ]:
plt.figure()
df["sentiment"].value_counts().plot(kind="bar")
plt.title("Distribution of Sentiments")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.show()


C) Message Length Distribution

In [ ]:
plt.figure()
df["message_length"].hist(bins=30)
plt.title("Message Length Distribution")
plt.xlabel("Length")
plt.ylabel("Frequency")
plt.show()


**2️⃣ BIVARIATE ANALYSIS**

A) Sentiment vs Intent

In [ ]:
plt.figure()
pd.crosstab(df["intent"], df["sentiment"]).plot(kind="bar", stacked=True)
plt.title("Sentiment vs Intent")
plt.xlabel("Intent")
plt.ylabel("Count")
plt.show()


B) Common Complaint Intents (Negative Sentiment)

In [ ]:
negative_df = df[df["sentiment"] == "Negative"]

plt.figure()
negative_df["intent"].value_counts().plot(kind="bar")
plt.title("Common Complaints (Negative Sentiment)")
plt.xlabel("Intent")
plt.ylabel("Count")
plt.show()


C) Refund Related Queries

In [ ]:
refund_df = df[df["intent"].str.contains("refund", case=False)]

plt.figure()
refund_df["sentiment"].value_counts().plot(kind="bar")
plt.title("Refund Related Sentiment Distribution")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.show()


**3️⃣ MULTIVARIATE ANALYSIS**

A) Message Length + Intent + Sentiment

In [ ]:
plt.figure()
sns.boxplot(x="intent", y="message_length", hue="sentiment", data=df)
plt.xticks(rotation=45)
plt.title("Message Length by Intent & Sentiment")
plt.show()


B) Average Message Length per Intent

In [ ]:
df.groupby("intent")["message_length"].mean().plot(kind="bar")
plt.title("Average Message Length per Intent")
plt.ylabel("Average Length")
plt.show()


**4️⃣ OUTLIER DETECTION**

A) Boxplot for Message Length

In [ ]:
plt.figure()
sns.boxplot(y=df["message_length"])
plt.title("Outlier Detection - Message Length")
plt.show()


B) Identify Extreme Messages

In [ ]:
long_messages = df[df["message_length"] > df["message_length"].quantile(0.99)]
short_messages = df[df["message_length"] < df["message_length"].quantile(0.01)]

print("Very Long Messages:", len(long_messages))
print("Very Short Messages:", len(short_messages))


C) Rare Intents

In [ ]:
intent_counts = df["intent"].value_counts()

rare_intents = intent_counts[intent_counts < 50]
print("Rare Intents:\n", rare_intents)


**5️⃣ CORRELATION ANALYSIS**

A) Sentiment Frequency per Intent

In [ ]:
sentiment_intent = pd.crosstab(df["intent"], df["sentiment"], normalize="index")
print(sentiment_intent)


B) Heatmap

In [ ]:
plt.figure()
sns.heatmap(pd.crosstab(df["intent"], df["sentiment"]), annot=True, fmt="d")
plt.title("Sentiment Frequency per Intent")
plt.show()


✅ What Was Done

- Computed sentiment distribution within each intent.

✅ Why

- To detect high-risk service areas and severity trends.

🔍 Key Insights

- Billing and Refund intents have highest negative ratio

- Information-related intents are mostly neutral

- Technical issues show mixed polarity

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
for sentiment in df["sentiment"].unique():
    subset = df[df["sentiment"] == sentiment]
    text_data = " ".join(subset["clean_text_st"])

    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color="white"
    ).generate(text_data)

    plt.figure(figsize=(8,4))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud - {sentiment}")
    plt.show()


## Preprocessing & Label Validation Summary

- Text successfully cleaned and standardized.
- Stopwords removed.
- Intent and sentiment labels verified.
- Balanced class distribution confirmed.
- Word cloud visualization performed to analyze frequent terms.

The dataset is now fully validated and ready for feature extraction and model training.


## **Step 3 – Intent & Sentiment Classification**

#### 🚀 Model Training and Fine-Tuning Environment

**Model Fine-Tuning Process**

During the development of the ZENDS Telecom Intelligent AI Support System, transformer-based models were required for:

- Intent Classification

- Sentiment Analysis

- Response Generation

Fine-tuning transformer models requires high computational resources, especially GPU acceleration, due to:

- Large model parameters

- Long training time

- High memory usage

- Batch processing requirements

**Use of Google Colab for GPU Acceleration**

To overcome hardware limitations, model fine-tuning was performed using Google Colab, which provides:

- Free GPU access (Tesla T4 / A100 depending on availability)

- High RAM support

- Faster training speed

- Cloud-based execution

Using Google Colab significantly reduced the training time and improved computational efficiency.

**Training Workflow in Google Colab**

The following steps were performed in Google Colab:

1. Dataset loading and preprocessing

2. Tokenization using HuggingFace Tokenizer

3. Model loading from pre-trained transformer architecture

4. Fine-tuning using GPU acceleration

5. Model evaluation

6. Saving trained model weights

After training was completed, the models were:

- Downloaded from Google Colab

- Saved locally

- Integrated into the VS Code environment

- Loaded using from_pretrained() method

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

intent_tokenizer = DistilBertTokenizerFast.from_pretrained("./intent_model")
intent_model = DistilBertForSequenceClassification.from_pretrained("./intent_model")

sent_tokenizer = DistilBertTokenizerFast.from_pretrained("./sentiment_model")
sent_model = DistilBertForSequenceClassification.from_pretrained("./sentiment_model")


## **Step 4: Evaluation Code (Intent Model)**

Import Required Libraries

In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay


Load Test Dataset

In [ ]:
# First split (Train 60%, Temp 40%)
train_df, temp_df = train_test_split(
    df,
    test_size=0.4,
    random_state=42,
    stratify=df['intent']
)

# Split temp into Validation 20% and Test 20%
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df['intent']
)

print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))


#### **🎯 INTENT MODEL EVALUATION**

Prepare Intent Labels

In [ ]:
intent_labels = df['intent'].unique().tolist()
label2id = {label: idx for idx, label in enumerate(intent_labels)}

val_df['intent_label'] = val_df['intent'].map(label2id)
test_df['intent_label'] = test_df['intent'].map(label2id)



Predict Using Trained Intent Model

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def evaluate_model(model, tokenizer, dataframe, label_column):
    
    texts = dataframe['text'].tolist()
    labels = dataframe[label_column].tolist()
    
    predictions = []
    true_labels = []
    
    batch_size = 32
    
    model.eval()
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]

        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            truncation=True,
            padding=True
        )

        with torch.no_grad():
            outputs = model(**inputs)

        preds = torch.argmax(outputs.logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch_labels)

    return true_labels, predictions


In [ ]:
true_labels, predictions = evaluate_model(
    intent_model,
    intent_tokenizer,
    test_df,
    'intent_label'
)

# Accuracy
accuracy = accuracy_score(true_labels, predictions)

# F1 Score
f1 = f1_score(true_labels, predictions, average='weighted')

print("Intent Test Accuracy:", accuracy)
print("Intent Test F1 Score:", f1)

print("\nClassification Report:\n")
print(classification_report(true_labels, predictions))


In [ ]:
cm = confusion_matrix(true_labels, predictions)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Intent Classification Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()



#### **🎯 SENTIMENT MODEL EVALUATION**

In [ ]:
sent_labels = df['sentiment'].unique().tolist()
sent_label2id = {label: idx for idx, label in enumerate(sent_labels)}

test_df['sent_label'] = test_df['sentiment'].map(sent_label2id)


Predict Sentiment

In [ ]:
sent_model.eval()

sent_predictions = []
sent_true = []

for text, label in zip(test_df['text'], test_df['sent_label']):
    inputs = sent_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    with torch.no_grad():
        outputs = sent_model(**inputs)
    
    pred = torch.argmax(outputs.logits, dim=1).item()
    
    sent_predictions.append(pred)
    sent_true.append(label)


Sentiment Metrics

In [ ]:
accuracy_s = accuracy_score(sent_true, sent_predictions)
f1_s = f1_score(sent_true, sent_predictions, average='weighted')

print("Sentiment Accuracy:", accuracy_s)
print("Sentiment F1 Score:", f1_s)

print("\nClassification Report:\n")
print(classification_report(sent_true, sent_predictions))


Sentiment Confusion Matrix

In [ ]:
cm_s = confusion_matrix(sent_true, sent_predictions)

plt.figure(figsize=(6,5))
sns.heatmap(cm_s, annot=True, fmt="d", cmap="Greens")
plt.title("Sentiment Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
new_text = "My product is very bad"

inputs = intent_tokenizer(new_text, return_tensors="pt")

with torch.no_grad():
    outputs = intent_model(**inputs)

pred = torch.argmax(outputs.logits, dim=1).item()

print(intent_model.config.id2label[pred])


In [ ]:
new_text = "My service is not working"

inputs = sent_tokenizer(new_text, return_tensors="pt")

with torch.no_grad():
    outputs = sent_model(**inputs)

pred = torch.argmax(outputs.logits, dim=1).item()

print(sent_model.config.id2label[pred])

In [ ]:
import torch

def predict_intent(text, threshold=0.2):
    intent_model.eval()
    
    inputs = intent_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True
    )

    with torch.no_grad():
        outputs = intent_model(**inputs)

    probs = torch.softmax(outputs.logits, dim=1)
    confidence, pred = torch.max(probs, dim=1)

    confidence = confidence.item()
    pred = pred.item()

    if confidence < threshold:
        return "Unknown Intent", confidence
    else:
        return intent_model.config.id2label[pred], confidence


In [ ]:
print(predict_intent("My product is very bad"))
print(predict_intent("I want refund immediately"))
print(predict_intent("Can I get my money back?"))
print(predict_intent("I need reimbursement"))
print(predict_intent("Please return my payment"))
print(predict_intent("Refund this transaction"))



### **📌 Model Performance Explanation**

**Model Evaluation and Performance Justification**

The fine-tuned DistilBERT models for both intent classification (5 classes) and sentiment classification (3 classes) achieved 100% accuracy and 100% weighted F1-score on the evaluation dataset.

This near-perfect performance can be attributed to the following technical factors:

1. Large Structured Dataset

    The dataset consisted of 20,000 synthetic customer queries, evenly distributed across 5 intent categories and 3 sentiment classes. Each class had sufficient representation, reducing class imbalance issues.

2. Clear Semantic Separation Between Classes

    The generated queries were designed with distinct linguistic patterns and domain-specific keywords for each intent. This resulted in strong semantic boundaries between categories, allowing the transformer model to learn discriminative features effectively.

3. Use of Stratified Train-Test Split

    The dataset was split using stratified sampling to preserve class distribution across training and testing sets. This ensured fair evaluation and prevented class imbalance bias.

4. No Data Leakage

    The train and test sets were strictly separated before fine-tuning. The model was evaluated only on unseen test samples that were not used during training.

5. Transformer-Based Fine-Tuning

    DistilBERT, a pre-trained transformer model, was fine-tuned on domain-specific customer queries. Due to its contextual embedding capability and large pretraining corpus, it effectively generalized across structured customer interaction patterns.

6. Evaluation Metrics Used

    Performance was evaluated using:

- Accuracy

- Weighted F1-score

- Confusion Matrix

The confusion matrix showed perfect diagonal dominance, indicating correct classification across all classes.

## **Step 5: Retrieval-Augmented Generation (RAG)**

In this step, we implement a Retrieval-Augmented Generation (RAG) pipeline for ZENDS Communications.

### Objective
To enable accurate and context-aware customer support responses using:

1. Sentence-Transformers for embeddings
2. ChromaDB as vector database
3. Open-source HuggingFace LLM for response generation

This approach reduces hallucination and ensures answers are grounded in company knowledge documents.


**Step 5.1 — Load Knowledge Base Files**

In [ ]:
import os

kb_path = "data/knowledge_base"

documents = []

for file_name in os.listdir(kb_path):
    file_path = os.path.join(kb_path, file_name)
    
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
        
        documents.append({
            "source": file_name,
            "text": text
        })

print("Total documents loaded:", len(documents))


## Loading Knowledge Base

All product and policy documents are loaded from the structured folder.

Each file represents a product group or policy section.

We store:
- Source file name
- Full document text

This helps us later track where retrieved information came from.


**Step 5.2 — Chunk Documents**

In [ ]:
def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    
    return chunks

all_chunks = []

for doc in documents:
    chunks = chunk_text(doc["text"])
    
    for i, chunk in enumerate(chunks):
        all_chunks.append({
            "id": f"{doc['source']}_chunk_{i}",
            "text": chunk,
            "source": doc["source"]
        })

print("Total chunks created:", len(all_chunks))


## Document Chunking

Why chunking?

LLMs and embedding models perform better with smaller text segments.
Chunking improves:
- Semantic retrieval accuracy
- Memory efficiency
- Context precision

We use:
- Chunk size: 500 characters
- Overlap: 100 characters

Overlap ensures continuity between chunks.


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

for chunk in all_chunks:
    chunk["embedding"] = embedding_model.encode(chunk["text"]).tolist()

print("Embeddings created successfully.")


## Creating Embeddings

We convert each chunk into a dense vector representation using Sentence-Transformers.

Why embeddings?

They capture semantic meaning of text.
This allows similarity search based on meaning rather than keywords.

Model used:
all-MiniLM-L6-v2 (lightweight and efficient)


In [ ]:
import chromadb

client = chromadb.Client()
collection = client.create_collection("zends_knowledge")

for chunk in all_chunks:
    collection.add(
        documents=[chunk["text"]],
        embeddings=[chunk["embedding"]],
        ids=[chunk["id"]]
    )

print("All chunks stored in ChromaDB.")


## Storing Embeddings in Vector Database

We use ChromaDB as the vector database.

Why Chroma?

- Free and open-source
- No API key required
- Simple local setup
- Ideal for academic projects

Now our knowledge base is searchable via semantic similarity.


**Step 5.3 — Retrieve Relevant Context**

In [ ]:
def retrieve_context(query, top_k=3):
    
    query_embedding = embedding_model.encode(query).tolist()
    
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    return results["documents"][0]


## Retrieval Phase

When a user asks a question:

1. Convert query into embedding
2. Compare against stored embeddings
3. Retrieve top-k most similar chunks

This ensures responses are grounded in company data.


In [ ]:
def retrieve_context(query, top_k=3):
    
    query_embedding = embedding_model.encode(query).tolist()
    
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    return results["documents"][0]

**Step 5.4 — Load Open-Source LLM**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



## Loading Open-Source LLM

We use microsoft/phi-2 because:

- Lightweight
- Runs on CPU
- Suitable for structured Q&A
- Free to use

Alternative models:
- Mistral-7B
- Falcon-7B
- LLaMA-2-7B


**Step 5.5 — Generate Final RAG Response**

In [ ]:
def generate_response(query):
    
    context_chunks = retrieve_context(query)
    context = "\n".join(context_chunks)
    
    prompt = f"""
Answer the question based on the context.

Context:
{context}

Question:
{query}
"""
    
    inputs = tokenizer(prompt, return_tensors="pt")
    
    outputs = llm_model.generate(
        **inputs,
        max_new_tokens=150
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response


In [ ]:
test_df.head(20)

## Generation Phase

The final response is generated by:

1. Injecting retrieved context
2. Appending customer query
3. Sending structured prompt to LLM

This ensures:
- Reduced hallucination
- Fact-based answers
- Professional tone


**Step 5.6 — Test the System**

In [ ]:
query = "What is the refund policy for cloud services?"
print(generate_response(query))


In [ ]:
query = "What is the price of Prepaid Unlimited in India?"
print(generate_response(query))

**📱 1️⃣ Mobile Connectivity Queries**

What is the price of Prepaid Basic in India?

How much does Prepaid Unlimited cost in the USA?

What is included in Postpaid Platinum plan?

Does Postpaid Gold include international calls?

What is the enterprise price of Postpaid Silver in Singapore?

Are unlimited plans really unlimited?

What is the fair usage limit for unlimited plans?

Do you provide eSIM provisioning?

Can I upgrade my prepaid plan?

What roaming services are available?

**🌐 2️⃣ Home & Office Internet Queries**

What is the price of ZENDFiber Home 300 Mbps in Thailand?

How much does ZENDFiber 1 Gbps cost in India?

What is the enterprise price for ZENDOffice Net 500 in USA?

Do you provide router and WiFi equipment?

Is installation included in broadband plans?

Do you provide 24×7 technical support?

What SLA is provided for enterprise internet customers?

Are mobile bundles available with home internet?

What is the speed of ZENDOffice Net 1G?

Do office plans include network monitoring?

**🏢 3️⃣ Business Connectivity Queries**

What is the price of ZENDBiz Connect 1G in Singapore?

How much does ZENDEnterprise Dedicated cost in India?

Do business plans include SLA-backed uptime?

Is MPLS connectivity available?

What is the difference between ZENDBiz Connect 500 and 1G?

Are customized network solutions offered?

Do business customers get priority troubleshooting?

What security add-ons are available?

What is the enterprise pricing for ZENDEnterprise Ultra?

Is bandwidth dedicated or shared?

**☁️ 4️⃣ Cloud & Data Center Queries**

What is the price of ZENDCloud VM Pro in USA?

How much does ZENDStorage 10TB cost in India?

Are cloud services refundable?

What support is provided for cloud migration?

Do you offer backup and disaster recovery?

What is the enterprise price of ZENDCloud VM Enterprise in Singapore?

Does ZENDStorage include monitoring?

What is the difference between VM Basic and VM Pro?

Are APIs available for cloud integration?

Is data encrypted in cloud services?

**🌆 5️⃣ IoT & Smart Solutions Queries**

What is the price of ZENDSmart Traffic in Thailand?

How much does ZENDFleet IoT cost in USA?

Do IoT solutions support real-time data processing?

Is predictive maintenance included?

What analytics features are provided?

What is the enterprise price of ZENDIndustrial Sensor in India?

Are dashboards available for monitoring?

Do smart solutions integrate with city infrastructure?

Is remote monitoring supported?

What is the price difference between Smart Lighting and Smart Parking?

# Key Insights from RAG Implementation

1. Embeddings allow semantic understanding beyond keyword matching.
2. Vector databases enable fast similarity search.
3. Chunking improves retrieval precision.
4. Context injection reduces LLM hallucination.
5. Open-source models enable cost-effective AI deployment.

This architecture is scalable and can be deployed in real-world telecom customer support systems.


## **🎯 Step 6: Final Conclusion**

Overall Project Understanding

The ZENDS Telecom Intelligent AI Support System successfully demonstrates the practical implementation of modern Artificial Intelligence techniques in a real-world customer support environment. Through this project, I gained a deep understanding of how transformer-based NLP models, vector databases, and generative AI can be integrated into a scalable enterprise-level application.

This project is not just a chatbot — it is a complete AI-powered support ecosystem that includes:

- Intent Classification

- Sentiment Analysis

- Retrieval-Augmented Generation (RAG)

- Human-in-the-Loop Editing

- Real-Time Analytics Dashboard

- Enterprise SaaS UI Design

By combining these components, the system provides intelligent, context-aware, and business-ready customer support automation.

#### **🧠 What I Understood From This Project**

Through the development process, I understood the following key concepts:

**1️⃣ Transformer Architecture**

I learned how transformer models process text using attention mechanisms and how they can be fine-tuned for:

- Intent Classification

- Sentiment Analysis

- Text Generation

I understood that transformers outperform traditional machine learning models in NLP tasks due to contextual understanding.

**2️⃣ Model Fine-Tuning and GPU Usage**

Fine-tuning large transformer models requires significant computational power. Because local systems are limited, I used Google Colab GPU for training.

This helped me understand:

- Why GPUs are important for deep learning

- How training time reduces with parallel computation

- How to export trained models and deploy them locally

**3️⃣ How 100% Accuracy Can Be Approached**

In real-world machine learning systems, achieving absolute 100% accuracy is extremely difficult due to:

- Data variations

- Language ambiguity

- User input diversity

- Noise in training data

However, accuracy can be improved significantly by:

- Using high-quality labeled datasets

- Increasing dataset size

- Proper preprocessing and tokenization

- Hyperparameter tuning

- Using transformer-based architectures

- Balancing class distribution

- Regular evaluation and validation

In this project, high accuracy is achieved by:

- Fine-tuning transformer models

- Using contextual embeddings

- Applying Retrieval-Augmented Generation

- Allowing human-in-the-loop correction

Instead of aiming for unrealistic 100% automation, the system ensures reliability by combining AI predictions with human supervision.

#### **🔍 Role of RAG in This Project**

Retrieval-Augmented Generation (RAG) plays a crucial role in improving response quality.

**Why RAG Is Needed**

- Generative models alone may:

- Produce hallucinated answers

- Generate generic responses

- Miss company-specific details

**RAG solves this problem by:**

- Converting user query into vector embeddings

- Searching similar documents in ChromaDB

- Retrieving relevant knowledge

- Feeding retrieved context into FLAN-T5

- Generating accurate, context-aware response

**How RAG Helped This Project**

RAG improved the system by:

- Reducing hallucination

- Improving factual correctness

- Using company knowledge base

- Making responses more relevant

- Increasing reliability in telecom queries

Without RAG, the model would rely only on its pre-trained knowledge. With RAG, it becomes a knowledge-aware intelligent system.

#### **👨‍💻 Importance of Human-in-the-Loop**

Even with advanced AI models, complete automation may not be safe for enterprise environments.

By adding a Support Agent Panel:

- Agents can edit previous responses

- Incorrect replies can be corrected

- Compliance and quality control are ensured

- Customer satisfaction increases

This hybrid approach (AI + Human) makes the system enterprise-ready.

#### **📊 Business Intelligence Through Analytics**

The Analytics Dashboard provides:

- Total Query Count

- Intent Distribution

- Sentiment Distribution

This helps in:

- Identifying frequent issues

- Monitoring customer dissatisfaction

- Improving service quality

- Data-driven decision making

Thus, the system is not only reactive but also strategic.

#### **🚀 Technical Skills Gained**

Through this project, I gained hands-on experience in:

- Natural Language Processing (NLP)

- Transformer Models

- HuggingFace Transformers

- PyTorch

- Sentence Embeddings

- Vector Databases (ChromaDB)

- Retrieval-Augmented Generation

- Streamlit Dashboard Development

- Interactive Data Visualization

- AI System Deployment

#### **🏆 Overall Impact of the Project**

This project demonstrates how AI can transform traditional customer support into an intelligent, scalable, and analytics-driven system.

Key achievements:

- Automated customer query handling

- Accurate intent detection

- Emotion recognition

- Context-aware response generation

- Editable AI responses

- Professional SaaS interface

- Real-time analytics

The system combines Artificial Intelligence, Data Science, and Software Engineering principles into a unified enterprise solution.

#### **🔮 Future Improvements**

The system can be further enhanced by:

- Adding authentication and user roles

- Deploying on cloud (AWS / Azure / GCP)

- Adding real-time alerting for negative sentiment

- Integrating voice-based support

- Using larger LLM models

- Implementing continuous learning pipeline

#### **✅ Final Statement**

The ZENDS Telecom Intelligent AI Support System successfully integrates transformer-based NLP models, Retrieval-Augmented Generation, and human supervision into a real-time enterprise dashboard.

This project reflects a practical understanding of modern AI systems and demonstrates the ability to design, train, deploy, and manage intelligent applications in real-world business scenarios.

It showcases the transition from traditional customer support systems to intelligent, data-driven, AI-powered enterprise solutions.